In [30]:
import sys
sys.path.append("..")

import itertools

import torch
import numpy as np
import matplotlib.pyplot as plt

from lra import temp_seed, ParityDataset, BinaryMarkovDataset, MajorityDataset, ListOpsTiny, IMDB, IMDBMini

In [31]:
class SinusoidalEncoding(torch.nn.Module):
    def __init__(self, dembed, denominator=10000.0):
        super().__init__()
        self.dembed = dembed
        self.denominator = denominator
        indexes = torch.arange(start=0, end=dembed, step=2) #[dmodel//2]
        div_term = denominator ** (indexes / dembed) #[dmodel//2]
        self.register_buffer('div_term', div_term)

    def forward(self, x, offset=0):
        #x : [..., seq_len]
        #returns : [batch, seq_len, dembed]
        positions = torch.arange(start=offset, end=offset+x.shape[1], step=1, device=x.device) #[seq_len]
        sin = torch.sin(positions.unsqueeze(-1) / self.div_term) #[seq_len, dembed//2]
        cos = torch.cos(positions.unsqueeze(-1) / self.div_term) #[seq_len, dembed//2]
        sin_cos = torch.stack([sin, cos], axis=-1).flatten(start_dim=-2)
        sin_cos = torch.broadcast_to(sin_cos, x.shape[:-1] + sin_cos.shape[-2:]) #[..., seq_len, dembed]
        return sin_cos


In [32]:
sinusoidal_encoding = SinusoidalEncoding(64)

In [34]:
x = torch.randn(1, 20)
plt.matshow(sinusoidal_encoding(x)[0])

NameError: name 'plt' is not defined

In [14]:
a = torch.tensor([[1, 2], [3, 4]])
b = torch.tensor([[4, 5], [6, 7]])

In [15]:
torch.stack([a, b], axis=-1).flatten(start_dim=-2)

tensor([[1, 4, 2, 5],
        [3, 6, 4, 7]])

In [ ]:
#positions : [batch, seq_len]
#returns : [batch, seq_len, dembed]

In [10]:
torch.arange(start=0, end=10, step=2).repeat_interleave(2)

tensor([0, 0, 2, 2, 4, 4, 6, 6, 8, 8])

In [3]:
dataset = IMDBMini('imdb')
dataset.setup()
next(iter(dataset.train_dataloader(batch_size=8)))

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/25000 [00:00<?, ? examples/s]

...

Map (num_proc=4):   0%|          | 0/25000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/25000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/25000 [00:00<?, ? examples/s]

IMDB char level | min_freq 15 | vocab size 120


(tensor([[49,  4, 25,  ...,  0,  0,  0],
         [37,  5, 11,  ...,  0,  0,  0],
         [45,  4, 29,  ...,  0,  0,  0],
         ...,
         [38, 19,  3,  ...,  0,  0,  0],
         [35,  3, 19,  ..., 23, 23,  2],
         [45,  4,  3,  ..., 20,  3,  2]]),
 tensor([0, 0, 0, 1, 0, 1, 0, 0]),
 {'lengths': tensor([ 662,  807,  329, 1000,  749,  618, 1000, 1000])})

In [3]:
lengths = [batch[2]['lengths'] for batch in dataset.train_dataloader(batch_size=32)]

In [4]:
lengths = torch.hstack(lengths)

In [5]:
lengths.float().max()

tensor(4096.)

In [12]:
lengths.float().max()

tensor(4096.)